<a href="https://colab.research.google.com/github/Yashgabani845/ml/blob/main/ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
# List the contents of the Google Drive root folder
os.listdir('/content/drive/MyDrive/')


['Getting started.pdf',
 'guj.jpeg',
 '12th.jpeg',
 'school lc.jpeg',
 'ViewGeneratedDocs.pdf',
 'school lc.pdf',
 '12th.pdf',
 'guj.pdf',
 'Classroom',
 'Session2_Assignment.gdoc',
 'Session4_find_command.gdoc',
 '16694875454677156463441062557371.jpg',
 '16694876457322312295232508056318.jpg',
 'Simple Filters (3).gdoc',
 'Simple Filters (2).gdoc',
 'Simple Filters (1).gdoc',
 'Simple Filters.gdoc',
 'BEE_LECTURE_TRANSFORMER CONT...gdoc',
 'Filters Using Regular Expressions-grep and sed.gdoc',
 'BEE (1).gdoc',
 'BEE.gdoc',
 'phy',
 'BTech_Sem II_CE_cultural_week_v1.xlsx',
 'iLovePDF',
 '22-11-2023_0302-0330.gdoc',
 'skrjgnerjg.gdoc',
 'LAB_06.pdf',
 'Untitled document (8).gdoc',
 'GSOC.gdoc',
 'Online_Stationery_Shop',
 'TechTitans-main',
 'Extension-hub',
 'GSOC Roadmap.gdoc',
 'Untitled document (7).gdoc',
 'Untitled document (6).gdoc',
 'rutu_gabani_resume.gdoc',
 'Untitled document (5).gdoc',
 'LAB01 (1).gdoc',
 'LAB01.gdoc',
 'Untitled document (4).gdoc',
 'lab2.gdoc',
 'GSSOC2024

In [ ]:
import zipfile
zip_path = '/content/drive/MyDrive/asl.zip'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/asl')



Start coding of ASL


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
# Image data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split dataset for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for test data

# Load training and validation data
train_data = train_datagen.flow_from_directory(
    '/content/asl/asl_aslphabet/asl_alphabet_train/asl_alphabet_train',  # Path to training data
    target_size=(64, 64),  # Resize images to 64x64
    batch_size=32,  # Use a batch size of 32
    class_mode='categorical',  # Since it's a classification task
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    '/content/asl/asl_aslphabet/asl_alphabet_train/asl_alphabet_train',  # Path to training data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [ ]:
# Update the CNN model to have 29 output neurons
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(29, activation='softmax')  # Change the number of classes here (29)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # Check the model architecture


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [ ]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,  # Number of epochs
    verbose=1
)


Epoch 1/5
  82/2175 [>.............................] - ETA: 7:47 - loss: 3.3724 - accuracy: 0.0396